# SQL

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Task 3

В этом упражнении ты создашь так называемую витрину данных. Она представляет собой таблицу, которую можно использовать для аналитических целей. Обычно она создается путем объединения нескольких отдельных таблиц. В этом упражнении мы будем собирать различные данные о наших пользователях: когда они сделали свои первые коммиты, когда они впервые посетили ленту новостей и т. д. Это поможет позднее выполнить анализ данных.

1.Создай соединение с базой данных с помощью библиотеки sqlite3.

In [ ]:
import sqlite3
path = "/content/drive/MyDrive/School21/Day10/src/data/checking-logs.sqlite"
connection = sqlite3.connect(path)
cursor = connection.cursor()

2.Создай новую таблицу datamart в базе данных, объединив таблицы pageviews и checker с помощью только одного запроса.  

Таблица должна содержать следующие столбцы: uid, labname, first_commit_ts, first_view_ts.  

first_commit_ts — это просто новое имя для столбца timestamp из таблицы checker; он показывает первый коммит конкретного лабораторного задания конкретного пользователя.  

first_view_ts — первое посещение пользователем из таблицы pageviews, метка времени посещения пользователем ленты новостей.  

По-прежнему нужно использовать фильтр status = 'ready'.  

По-прежнему нужно использовать фильтр numTrials = 1.  

Имена лабораторных заданий по-прежнему должны быть из следующего списка: laba04, laba04s, laba05, laba06, laba06s, project1.  

Таблица должна содержать только пользователей (uid с user_*), а не администраторов.  

first_commit_ts и first_view_ts должны быть распарсены как datetime64[ns].

In [ ]:
import pandas as pd

zapros = f"""SELECT s1.uid, s1.labname, s1.first_commit_ts, s2.first_view_ts
FROM (SELECT uid, labname, timestamp AS first_commit_ts FROM checker
WHERE uid LIKE 'user_%'
AND status = 'ready'
AND numTrials = 1
AND labname IN ('laba04', 'laba04s', 'laba05', 'laba06', 'laba06s', 'project1')
GROUP BY uid, labname) AS s1 LEFT JOIN (SELECT uid, MIN(datetime) AS first_view_ts
FROM pageviews
WHERE uid LIKE 'user_%'
GROUP BY uid)
AS s2 ON s1.uid = s2.uid;"""

dfDatamart = pd.io.sql.read_sql(zapros, con=connection,  parse_dates=['first_commit_ts','first_view_ts'])
dfDatamart.to_sql('datamart', connection, if_exists='replace',index=False)

dfDatamart.info()
dfDatamart


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140 entries, 0 to 139
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              140 non-null    object        
 1   labname          140 non-null    object        
 2   first_commit_ts  140 non-null    datetime64[ns]
 3   first_view_ts    59 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 4.5+ KB


,uid,labname,first_commit_ts,first_view_ts
0,user_1,laba04,2020-04-26 17:06:18.462708,2020-04-26 21:53:59.624136
1,user_1,laba04s,2020-04-26 17:12:11.843671,2020-04-26 21:53:59.624136
2,user_1,laba05,2020-05-02 19:15:18.540185,2020-04-26 21:53:59.624136
3,user_1,laba06,2020-05-17 16:26:35.268534,2020-04-26 21:53:59.624136
4,user_1,laba06s,2020-05-20 12:23:37.289724,2020-04-26 21:53:59.624136
...,...,...,...,...
135,user_8,laba04s,2020-04-19 10:22:35.761944,NaT
136,user_8,laba05,2020-05-02 13:28:07.705193,NaT
137,user_8,laba06,2020-05-16 17:56:15.755553,NaT
138,user_8,laba06s,2020-05-16 20:01:07.900727,NaT


3.Используя методы библиотеки Pandas, создай два датафрейма: test и control.  

test должен включать пользователей, у которых имеются значения в first_view_ts.  

control должен включать пользователей, у которых отсутствуют значения в first_view_ts.  

Замени пропущенные значения в control средним значением first_view_ts пользователей из test (оно пригодится нам для анализа в будущем).  

Сохрани обе таблицы в базе данных (вы будете использовать их в следующих упражнениях).

In [ ]:
pd.options.mode.chained_assignment = None # убрать варнинг
test = dfDatamart[dfDatamart['first_view_ts'].notnull()]
control = dfDatamart[dfDatamart['first_view_ts'].isnull()]
meanVals = test['first_view_ts'].mean()
control['first_view_ts'].fillna(meanVals, inplace=True)
control.to_sql('control', connection, if_exists='replace',index=False)
test.to_sql('test', connection, if_exists='replace',index=False)

test.info()
control.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 59 entries, 0 to 114
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              59 non-null     object        
 1   labname          59 non-null     object        
 2   first_commit_ts  59 non-null     datetime64[ns]
 3   first_view_ts    59 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 2.3+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 81 entries, 12 to 139
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              81 non-null     object        
 1   labname          81 non-null     object        
 2   first_commit_ts  81 non-null     datetime64[ns]
 3   first_view_ts    81 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 3.2+ KB


In [ ]:
pd.io.sql.read_sql('SELECT * FROM control', connection)

,uid,labname,first_commit_ts,first_view_ts
0,user_11,laba05,2020-05-03 21:06:55.970293,2020-04-27 00:40:05.761783
1,user_11,project1,2020-05-03 23:45:33.673409,2020-04-27 00:40:05.761783
2,user_12,laba04,2020-04-18 17:07:51.767358,2020-04-27 00:40:05.761783
3,user_12,laba04s,2020-04-26 15:42:38.070593,2020-04-27 00:40:05.761783
4,user_12,laba05,2020-05-03 08:39:25.174316,2020-04-27 00:40:05.761783
...,...,...,...,...
76,user_8,laba04s,2020-04-19 10:22:35.761944,2020-04-27 00:40:05.761783
77,user_8,laba05,2020-05-02 13:28:07.705193,2020-04-27 00:40:05.761783
78,user_8,laba06,2020-05-16 17:56:15.755553,2020-04-27 00:40:05.761783
79,user_8,laba06s,2020-05-16 20:01:07.900727,2020-04-27 00:40:05.761783


4.Закрой соединение с базой данных.

In [ ]:
connection.close()